In [14]:
import numpy as np 

In [15]:
def function(x):
    '''
    функция интегрального синуса 
    '''
    if x == 0: 
        return 1
    else:
        return np.sin(x) / x

In [25]:
def Mid_rect(a:float, b:float, n:int) -> float:
    """
    Метод центральных прямоугольников:
    В приближённом вычислении площади под графиком суммированием площадей 
    конечного числа прямоугольников, ширина которых будет определяться 
    расстоянием между соответствующими соседними узлами интегрирования, 
    а высота — значением подынтегральной функции в этих узлах.
    
    Расчет интеграла производится по формуле:
    h*f(a + b/2), где h = (b-a)/n
    
    Ключевые параметры:
    a: type float: начальная точка
    b: type float: конечная точка
    n: type int: значение общей точности шагов
    
    Возращает:
    : type float, интегральная сумма
    """
    h = (b-a)/n
    z = [a + i*h for i in np.arange(0, n+1)]
    temp = [function((z[i-1] + z[i])*0.5) for i in np.arange(1, n+1)]
    
    return h * np.sum(temp)

In [27]:
def Simpson(a:float, b:float, n:int) -> float:
    """
    Метод Симсона(парабол):
    Использовав три точки отрезка интегрирования, 
    можно заменить подынтегральную функцию параболой. 
    Обычно в качестве таких точек используют концы отрезка и его среднюю точку
    
    Расчет интеграла производится по формуле:
    b-a/6*[f(a)+4f(a+b/2)+f(b)]
    
    Ключевые параметры:
    a: type float: начальная точка
    b: type float: конечная точка
    n: type int: значение общей точности шагов
    
    Возращает:
    : type float, интегральная сумма
    """
    h = (b-a)/n
    z = [a + i*h for i in np.arange(0, n+1)]
    temp = [function(z[i-1]) + 4*function(z[i-1] + h*0.5) + function(z[i]) \
            for i in np.arange(1, n+1)]
    
    return (h/6) * np.sum(temp)

In [18]:
def Gauss(a:float, b:float, n:int) -> float:
    """
    Метод Гаусса:
    Позволяет повысить алгебраический порядок точности методов 
    на основе интерполяционных формул путём специального выбора 
    узлов интегрирования без увеличения числа используемых 
    значений подынтегральной функции. 
    h/2 * [f(b + h/2*(1-1/sqrt(3)))) f(b + h/2*(1+1/sqrt(3))))], где h = (b-a)/n 
    
    Ключевые параметры:
    a: type float: начальная точка
    b: type float: конечная точка
    n: type int: значение общей точности шагов
    
    Возращает:
    : type float, интегральная сумма
    """
    h = (b-a)/n
    z = [a + i*h for i in np.arange(0, n+1)]
    temp = [function(z[i-1] + (h / 2)*(1 - 1 / 3**0.5))\
            + function(z[i-1] + (h / 2)*(1 + 1 / 3**0.5)) for i in np.arange(1, n+1) ]
    
    return (h / 2) * np.sum(temp) 

In [19]:
def Trapezium(a:float, b:float, n:int) -> float:
    """
    Метод трапеций:
    Вычисляет интегральную сумму путем деления на прямоугольники, 
    равные по x, и суммирует их, принимая за значение функции 
    среднее значение между предыдущими x и x + delta(x) 
    вертикальной стороны прямоугольника с высотой f(x).
    h/2 * (f(b) + f(a)), где  h = (b-a)/n
    
    Ключевые параметры:
    a: type float: начальная точка
    b: type float: конечная точка
    n: type int: значение общей точности шагов
    
    Возращает:
    : type float, интегральная сумма
    """
    h = (b-a)/n
    z = [a + i*h for i in np.arange(0, n+1)]
    temp = [function(z[i-1]) + function(z[i]) for i in np.arange(1, n+1)]    
    
    return (h/2) * np.sum(temp)

In [20]:
def runge(previous, current, theta):
    '''
    оценивает погрешность для шага 2n
    для данного theta по правилу Рунге
    '''
    return np.abs(current - previous) * theta

In [21]:
def Quadrature(a:float, b:float, eps:float, method:str) -> (float, int):
    """
    квадратурные формулы для оценки значения определённого интеграла
    
    Ключевые параметры:
    a: type float: начальная точка
    b: type float: конечная точка
    eps: type float: точность
    method: type str: название метода 
    
    Возращает:
    current: type float: интегральная сумма
    n: type int: число разбиений 
    """
    if method == "Гаусс":
        rule = Gauss
        theta=1
    elif method =="Симпсон":
        rule = Simpson
        theta=15
    elif method =="Центральных_Прямоугольников":
        rule = Mid_rect
        theta=3
    elif method == "Трапеций":
        rule = Trapezium
        theta=3

        
    n = 1
    current = rule(a, b, n)
    error = np.inf
    while error > eps:
        n *= 2
        previous, current = current, rule(a, b, n)
        error = runge(previous, current, theta)
        
    return current, n

In [22]:
def Si(x, n=1000, t = 0, eps=1e-10):
    """
    Интегрального синуса основанный на ряде Тейлора
    
    Ключевые параметры:
    x: type float: до какой точки интегрируем 
    n: type float: количество разбиений
    t: type int: счетчик, начинаем разбиение с 0
    eps: type float: точность

    
    Возращает:
    result: type float: Q - десятичное приближение 
    """

    Q = np.float32(0)
    for t in np.arange(0, n):
        temp = (-1)**t * x**(2*t+1) / (np.math.factorial(2*t+1)*(2*t+1))
        Q += temp
        
        if (np.abs(temp) <= eps): break
             
    return Q

In [23]:
def Table():
    '''
    построения таблицы с результатами 
    '''
    print("Вычисление интеграла")
    h = np.arange(0, 4, 0.40)
    h = np.append(h, 4)
    epsilons = [1e-6, 1e-8]
    methods = ["Гаусс","Центральных_Прямоугольников", "Симпсон", "Трапеций"]
    for method in methods:
        print(method)
        
        for eps in epsilons:
            print(eps)
            print(f"{'X':>5}{'S(X)':>10}{'N':>10}{'error':>20}")
            
            for i in range(0, len(h)):
                result, k  = Quadrature(h[0], h[i], eps, method)
                forError = abs(Si(h[i]) - result)
                print(f"{h[i]:>5.2f}{result:>10.5f}{k:>10}{forError:>20.8f}")
                
            print("-----------------------------------------------")

In [28]:
Table()

Вычисление интеграла
Гаусс
1e-06
    X      S(X)         N               error
 0.00   0.00000         2          0.00000000
 0.40   0.39646         2          0.00000003
 0.80   0.77210         4          0.00000005
 1.20   1.10805         8          0.00000002
 1.60   1.38918        16          0.00000001
 2.00   1.60541        16          0.00000001
 2.40   1.75249        16          0.00000003
 2.80   1.83210        16          0.00000004
 3.20   1.85140        16          0.00000004
 3.60   1.82195        16          0.00000003
 4.00   1.75820        16          0.00000002
-----------------------------------------------
1e-08
    X      S(X)         N               error
 0.00   0.00000         2          0.00000000
 0.40   0.39646         8          0.00000000
 0.80   0.77210        16          0.00000000
 1.20   1.10805        32          0.00000000
 1.60   1.38918        32          0.00000000
 2.00   1.60541        64          0.00000000
 2.40   1.75249        64          0.00